# Load Using MS Learn Script

1. Run this cell Below

Reference: 
- https://learn.microsoft.com/pt-br/fabric/data-science/how-to-consume-data-agent
- https://learn.microsoft.com/pt-br/fabric/data-science/data-agent-scenario


In [1]:
import pandas as pd
from tqdm.auto import tqdm
base = "https://synapseaisolutionsa.blob.core.windows.net/public/AdventureWorks"

# load list of tables
df_tables = pd.read_csv(f"{base}/adventureworks.csv", names=["table"])

for table in (pbar := tqdm(df_tables['table'].values)):
    pbar.set_description(f"Uploading {table} to lakehouse")

    # download
    df = pd.read_parquet(f"{base}/{table}.parquet")

    # save as lakehouse table
    spark.createDataFrame(df).write.mode('overwrite').saveAsTable(table)

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 3, Finished, Available, Finished)

  0%|          | 0/28 [00:00<?, ?it/s]

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 4, Finished, Available, Finished)

In [6]:
import pandas as pd
from tqdm.auto import tqdm
base = "https://synapseaisolutionsa.blob.core.windows.net/public/AdventureWorks"

# load list of tables
df_tables = pd.read_csv(f"{base}/adventureworks.csv", names=["table"])

for table in (pbar := tqdm(df_tables['table'].values)):
    pbar.set_description(f"Uploading {table} to lakehouse")

    # download
    df = pd.read_parquet(f"{base}/{table}.parquet")

    # save as lakehouse table
    spark.createDataFrame(df).write.mode('overwrite')\
          .option("overwriteSchema", "true")\
          .save(f"abfss://fabric_aiskills_demo@onelake.dfs.fabric.microsoft.com/AdvWorksLH.Lakehouse/Files/Data/{table}")

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 11, Finished, Available, Finished)

  0%|          | 0/28 [00:00<?, ?it/s]

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 12, Finished, Available, Finished)

# Load Manually files to Lakehouse

1. Upload Parquet Files
2. Run Cell Below

In [ ]:
# Load Lakehouse Files to Tables if they are uploaded manually. 
import os
from pyspark.sql import SparkSession
from notebookutils import mssparkutils
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

# Initialize SparkSession
spark = SparkSession.builder.appName("LoadAllParquetFiles").getOrCreate()

# Define the directory containing the files #Short Path for Spark
directory_path = "Files"

# Get the list of all files in the directory
file_list = [item.name for item in mssparkutils.fs.ls(directory_path)]

for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)
    
    # Check if the file is a Parquet file
    if file_name.endswith('.parquet'):
        try:
            # Create a DataFrame from the Parquet file
            df = spark.read.format("parquet").load(file_path)
            
            # Display the DataFrame (to ensure data is being loaded correctly)
            # df.show()
            
            # Generate table name by removing file extension and replacing spaces/dashes with underscores
            table_name = os.path.splitext(file_name)[0].replace(' ', '_').replace('-', '_')
            
            # Save the DataFrame to the lakehouse table
            df.write.format("delta").saveAsTable(table_name)
        
        except Exception as e:
            print(f"Failed to load {file_name}. Error: {str(e)}")